In [1]:
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

import dn3.configuratron
print(dir(dn3.configuratron))

C:\Users\chedl\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['DatasetConfig', 'ExperimentConfig', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'config', 'extensions']


In [3]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne
mne.set_log_level(False)

In [25]:
config_filename = 'my_config.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['mmidb']
dataset = ds_config.auto_construct_dataset()

Adding additional configuration entries: dict_keys(['train_params'])
Configuratron found 1 datasets.


Scanning C:\Users\chedl\OneDrive\Bureau\EMSE\3A\PE\files. If there are a lot of files, this may take a while...: 100%|██████████| 4/4 [00:00<00:00, 26.23it/s, extension=.gdf]


Creating dataset of 315 Preloaded Epoched recordings from 105 people.


Loading Physionet MMIDB: 100%|██████████| 105/105 [00:09<00:00, 11.15person/s]


>> Physionet MMIDB | DSID: None | 105 people | 4408 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Constructed 1 channel maps
Used by 315 recordings:
EEG (original(new)): Fc5.(FC5) Fc3.(FC3) Fc1.(FC1) Fcz.(FCZ) Fc2.(FC2) Fc4.(FC4) Fc6.(FC6) C5..(C5) C3..(C3) C1..(C1) Cz..(CZ) C2..(C2) C4..(C4) C6..(C6) Cp5.(CP5) Cp3.(CP3) Cp1.(CP1) Cpz.(CPZ) Cp2.(CP2) Cp4.(CP4) Cp6.(CP6) Fp1.(FP1) Fpz.(FPZ) Fp2.(FP2) Af7.(AF7) Af3.(AF3) Afz.(AFZ) Af4.(AF4) Af8.(AF8) F7..(F7) F5..(F5) F3..(F3) F1..(F1) Fz..(FZ) F2..(F2) F4..(F4) F6..(F6) F8..(F8) Ft7.(FT7) Ft8.(FT8) T7..(T7) T8..(T8) T9..(T9) T10.(T10) Tp7.(TP7) Tp8.(TP8) P7..(P7) P5..(P5) P3..(P3) P1..(P1) Pz..(PZ) P2..(P2) P4..(P4) P6..(P6) P8..(P8) Po7.(PO7) Po3.(PO3) Poz.(POZ) Po4.(PO4) Po8.(PO8) O1..(O1) Oz..(OZ) O2..(O2) Iz..(IZ) 
EOG (original(new)): 
REF (original(new)): 
EXTRA (original(new)): 
Heuristically Assigned: Fc5.(FC5)  Fc3.(FC3)  Fc1.(FC1)  Fcz.(FCZ)  Fc2.(FC2)  Fc4.(FC4)  Fc6.(FC6)  C5..(C5)  C3..(C3)  C1..(C1)  Cz

In [29]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.train_params.lr)

In [33]:
results = list()
for training, validation, test in dataset.lmso(ds_config.train_params.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation)

    results.append(process.evaluate(test)['Accuracy'])

print(results)
print("Average accuracy: {:.2%}".format(sum(results)/len(results)))

Training:   >> Physionet MMIDB | DSID: None | 63 people | 2646 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Validation: >> Physionet MMIDB | DSID: None | 21 people | 880 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Test:       >> Physionet MMIDB | DSID: None | 21 people | 882 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Creating TIDNet using: 90 channels x 960 samples at 160.0Hz | 2 targets
Loading data with 0 additional workers


Epoch: 100%|##########| 1/1 [00:00<?, ?epoch/s]

Iteration:   0%|          | 1/330 [00:00<?, ?batches/s]

Training: End of Epoch 1 | Accuracy: 67.65% | loss: 0.633 | lr: 5.024e-05 | momentum: 0.900 | epoch: 1.000 |


Predicting:   0%|          | 0/110 [00:00<?, ?it/s]

...

Training: End of Epoch 1 | Accuracy: 67.65% | loss: 0.627 | lr: 5.024e-05 | momentum: 0.900 | epoch: 1.000 |


Predicting:   0%|          | 0/111 [00:00<?, ?it/s]

Validation: End of Epoch 1 | Accuracy: 76.76% | loss: 0.464 |
Best loss. Retaining checkpoint...
Loading best model...


Predicting:   0%|          | 0/880 [00:00<?, ?it/s]

[0.8378684520721436, 0.7947846055030823, 0.8492063283920288, 0.774376392364502, 0.7954545617103577]
Average accuracy: 81.03%


In [34]:
results = dict()
for training, validation, test in dataset.lmso(experiment.training_configuration.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation,
                epochs=experiment.training_configuration.epochs,
                batch_size=experiment.training_configuration.batch_size)

    for _, _, test_thinker in test.loso():
        results[test_thinker.person_id] = process.evaluate(test_thinker)

print(results)
print("Average accuracy: {:.2%}".format(sum(results.values())/len(results)))

AttributeError: 'ExperimentConfig' object has no attribute 'training_configuration'